In [1]:
from Cartan import *
import numpy as np
import scipy.optimize as opt

In [2]:
class itCartan(Cartan, Pauli):
    
    
    def __init__(self, N, model = None):
        Cartan.__init__(self, N, model)
        Pauli.__init__(self)
        
        
    def Abelian(self):
        
        b = self.subalgebra()
        i = 0
        
        while i < len(b):
            j = i + 1
            
            while j < len(b):
                x = ()
                
                for k in range(self.N):
                    x += (self.product(b[i][k], b[j][k]),)

                if x in b:
                    index = b.index(x)
                    b.remove(x)
                    
                    if index > j:
                        j += 1
                        
                    elif index < i:
                        i -= 1

                else:
                    j += 1

            i += 1

        return b


    def subspace(self):

        k = self.decomposition()[0]
        b = self.Abelian()
        k_r = [[],]*len(b)

        for i in range(len(b)):
            y = []
            j = 0
            
            while j < len(k):

                A = self.commutator(b[i],k[j])
                
                if A != 0:
                    y.append(k[j])
                    k.pop(j)

                else:
                    j += 1

            k_r[i] = y

        return k_r      

In [3]:
class parameters(itCartan, functions):


    def __init__(self, N, model = None):
        itCartan.__init__(self, N, model)
        functions.__init__(self)

    

    def find(self, coeff = None):

        H = self.Hamiltonian(coeff)
        Hp = self.prepare(H[1], H[0])
        K = self.subspace()
        B = self.Abelian()
        p = []
        #bound = np.pi

        while K[-1] == []:
            K.pop(-1)
            B.pop(-1)

            if K == []:
                break

        for i in range(len(B)):

            Bp = self.prepare(1, B[i])

            def f_r(a):

                Xp = self.KXK([a[j] for j in range(len(K[i]))], K[i], Bp)
                Fp = self.fullprod(Xp, Hp)
                return np.real(self.trace(Fp))

            a0 = 2*np.pi*np.random.rand(len(K[i]))
            #bounds = ((-bound, bound),)*len(a0)
            
            result = opt.minimize(f_r, a0, method="BFGS")

            a = result.x % (2*np.pi)
            p = p + list(a)

            c = list(-a)
            c.reverse()
            K[i].reverse()

            Hp = self.KXK(c, K[i], Hp)

        i = 0
        while i < len(Hp):

            if abs(Hp[i][0]) < 1e-4:
                Hp.pop(i)

            else:
                i += 1
            

        return p, Hp
            

In [4]:
p = parameters(4, "TFIM").find(coeff = None)
angles = p[0]
Hamiltonian = p[1]
print(angles)
Pauli().letters(Hamiltonian)

[0.9734628688529193, 5.2086490606994476, 3.4589640082443167, 5.876478912435575, 4.933818453380409, 6.1681663963015705, 5.360336451993548, 4.173612574048753, 1.342930482710869, 2.7348848993189256, 0.9734607514852883, 3.5022487208508597]
[((-0.34729635534707737+0j), 'ZIII'), ((-0.9999999999861269+0j), 'IZII'), ((-1.532088886235989+0j), 'IIZI'), ((-1.8793852415661005+0j), 'IIIZ')]


In [5]:
model = itCartan(4, "TFIM")
print("4-site TFIM \n ")
print("H: ")
Pauli().letters(model.Hamiltonian()[0])
print("\ng: ")
Pauli().letters(model.algebra()[0])
print("\nk: ")
Pauli().letters(model.decomposition()[0])
print("\nm: ")
Pauli().letters(model.decomposition()[1])
print("\nh: ")
Pauli().letters(model.subalgebra())
print("\nb: ")
Pauli().letters(model.Abelian())
k = model.subspace()
for i in range(len(k)):
    print(f"\nk%i: " %(i+1))
    Pauli().letters(model.subspace()[i])

4-site TFIM 
 
H: 
['XXII', 'ZIII', 'IXXI', 'IZII', 'IIXX', 'IIZI', 'IIIZ']

g: 
['XXII', 'ZIII', 'IXXI', 'IZII', 'IIXX', 'IIZI', 'IIIZ', 'IIXY', 'IIYX', 'IXYI', 'IYXI', 'XYII', 'YXII', 'YZXI', 'YYII', 'XZYI', 'XZXI', 'IYZX', 'IYYI', 'IXZY', 'IXZX', 'IIYY', 'XZZY', 'YZZY', 'IYZY', 'YZZX', 'XZZX', 'YZYI']

k: 
['IIXY', 'IIYX', 'IXYI', 'IYXI', 'XYII', 'YXII', 'YZXI', 'XZYI', 'IYZX', 'IXZY', 'XZZY', 'YZZX']

m: 
['XXII', 'ZIII', 'IXXI', 'IZII', 'IIXX', 'IIZI', 'IIIZ', 'YYII', 'XZXI', 'IYYI', 'IXZX', 'IIYY', 'YZZY', 'IYZY', 'XZZX', 'YZYI']

h: 
['IIIZ', 'IIZI', 'IZII', 'ZIII']

b: 
['IIIZ', 'IIZI', 'IZII', 'ZIII']

k1: 
['IIXY', 'IIYX', 'IYZX', 'IXZY', 'XZZY', 'YZZX']

k2: 
['IXYI', 'IYXI', 'YZXI', 'XZYI']

k3: 
['XYII', 'YXII']

k4: 
[]
